# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [5]:
# Get some information of the datasets
school_data_complete.head()
school_data_complete.dtypes

school_data.head()
school_data.dtypes

student_data.head()
student_data.dtypes

Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
dtype: object

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [15]:
# School Names and Count
school_names = school_data['school_name'].unique()
school_count = len(school_names)

# Distract Sum and Count
dist_sum = school_data['size'].sum()
dist_count = student_data['student_name'].count()

# Budget
total_budget = school_data['budget'].sum()

# Reading Number and Percentage
num_reading = student_data.loc[student_data['reading_score'] >= 70]['reading_score'].count()
perc_reading = num_reading/dist_count * 100

# Math Number and Percentage
num_math = student_data.loc[student_data['math_score'] >= 70]['math_score'].count()
perc_math = num_math/dist_count * 100

# Average Reading
avg_reading_score = student_data['reading_score'].mean()

# Average Math
avg_math_score = student_data['math_score'].mean()

# Total Passing Rate
total_pass = (avg_reading_score + avg_math_score) / 2

# Distract Summary DataFrame using Pandas
district_summary = pd.DataFrame({
    
    "Total Schools": [school_count],
    "Total Students": [dist_sum],
    "Total Budget": [total_budget],
    "Average Reading Score": [avg_reading_score],
    "Average Math Score": [avg_math_score],
    "% Passing Reading":[perc_reading],
    "% Passing Math": [perc_math],
    "% Overall Passing Rate": [total_pass]
    })

# Clean up the Distract Summary DataFrame
district_summary = district_summary[["Total Schools", "Total Students","Total Budget",
                                     "Average Math Score", "Average Reading Score",
                                     "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]

# Format the outputs using .map().format
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.2f}%".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.2f}%".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}%".format)
district_summary["% Overall Passing Rate"] = district_summary["% Overall Passing Rate"].map("{:.2f}%".format)


district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.99%,81.88%,74.98%,85.81%,80.43%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [69]:
# School Type
school_type = school_data.set_index(["school_name"])["type"]

# Total Students
school_students = school_data_complete["school_name"].value_counts()

# Total School Budget
school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
school_total_budget = school_budget / school_students

# Average Math Scores
perc_school_math = school_data_complete.groupby(["school_name"]).mean()["math_score"]

# Average Readings Scores
perc_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]

# Passing Scores for Math
school_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)]

# Passing Scores for Reading
school_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]

perc_school_passing_math = (school_passing_math.groupby(["school_name"]).count()["student_name"] / school_students) * 100
perc_school_passing_reading = (school_passing_reading.groupby(["school_name"]).count()["student_name"] / school_students) * 100
overall_passing_rate = (perc_school_passing_math + perc_school_passing_reading) / 2

# Convert to data frame
perc_school_summary = pd.DataFrame({
    "School Type": school_type,
    "Total Students": school_students,
    "Total School Budget": school_budget,
    "Per Student Budget": school_total_budget,
    "Average Math Score": perc_school_math,
    "Average Reading Score": perc_school_reading,
    "% Passing Math": perc_school_passing_math,
    "% Passing Reading": perc_school_passing_reading,
    "% Overall Passing Rate": overall_passing_rate
    })

# Minor data munging
school_summary = perc_school_summary[["School Type", "Total Students", "Total School Budget", "Per Student Budget",
                                         "Average Math Score", "Average Reading Score", 
                                         "% Passing Math", "% Passing Reading", 
                                         "% Overall Passing Rate"]]

# Format Data
school_summary["Total Students"] = school_summary["Total Students"].map("{:,}".format)
# school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
# Why does this format return an error
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)
school_summary["Average Math Score"] = school_summary["Average Math Score"].map("{:.2f}%".format)
school_summary["Average Reading Score"] = school_summary["Average Reading Score"].map("{:.2f}%".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{:.2f}%".format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map("{:.2f}%".format)
school_summary["% Overall Passing Rate"] = school_summary["% Overall Passing Rate"].map("{:.2f}%".format)

# Display the data frame
school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,"4,976",3124928.0,$628.00,77.05%,81.03%,66.68%,81.93%,74.31%
Cabrera High School,Charter,"1,858",1081356.0,$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Figueroa High School,District,"2,949",1884411.0,$639.00,76.71%,81.16%,65.99%,80.74%,73.36%
Ford High School,District,"2,739",1763916.0,$644.00,77.10%,80.75%,68.31%,79.30%,73.80%
Griffin High School,Charter,"1,468",917500.0,$625.00,83.35%,83.82%,93.39%,97.14%,95.27%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [73]:
# Display top five schools, sort by overall passing rate, format, display df.head()
top_five_schools = perc_school_summary.sort_values(["% Overall Passing Rate"], ascending=False)

top_five_schools["Total Students"] = top_five_schools["Total Students"].map("{:,}".format)
top_five_schools["Total School Budget"] = top_five_schools["Total School Budget"].map("${:,.2f}".format)
top_five_schools["Per Student Budget"] = top_five_schools["Per Student Budget"].map("${:,.2f}".format)
top_five_schools["Average Math Score"] = top_five_schools["Average Math Score"].map("{:.2f}%".format)
top_five_schools["Average Reading Score"] = top_five_schools["Average Reading Score"].map("{:.2f}%".format)
top_five_schools["% Passing Math"] = top_five_schools["% Passing Math"].map("{:.2f}%".format)
top_five_schools["% Passing Reading"] = top_five_schools["% Passing Reading"].map("{:.2f}%".format)
top_five_schools["% Overall Passing Rate"] = top_five_schools["% Overall Passing Rate"].map("{:.2f}%".format)

top_five_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42%,83.85%,93.27%,97.31%,95.29%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84%,84.04%,94.59%,95.95%,95.27%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27%,83.99%,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [75]:
# Display bottom five schools, sort by overall passing rate, display df.head(), change ascending=True
bottom_five_schools = perc_school_summary.sort_values(["% Overall Passing Rate"], ascending=True)

bottom_five_schools["Total Students"] = bottom_five_schools["Total Students"].map("{:,}".format)
bottom_five_schools["Total School Budget"] = bottom_five_schools["Total School Budget"].map("${:,.2f}".format)
bottom_five_schools["Per Student Budget"] = bottom_five_schools["Per Student Budget"].map("${:,.2f}".format)
bottom_five_schools["Average Math Score"] = bottom_five_schools["Average Math Score"].map("{:.2f}%".format)
bottom_five_schools["Average Reading Score"] = bottom_five_schools["Average Reading Score"].map("{:.2f}%".format)
bottom_five_schools["% Passing Math"] = bottom_five_schools["% Passing Math"].map("{:.2f}%".format)
bottom_five_schools["% Passing Reading"] = bottom_five_schools["% Passing Reading"].map("{:.2f}%".format)
bottom_five_schools["% Overall Passing Rate"] = bottom_five_schools["% Overall Passing Rate"].map("{:.2f}%".format)

bottom_five_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84%,80.74%,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63%,81.18%,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07%,80.97%,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10%,80.75%,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [33]:
# Breakdown school data by 9th, 10th, 11th, 12th
ninth = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth = school_data_complete[(school_data_complete["grade"] == "12th")]

# Using the variables created in the above code, create 9th, 10th, 11th, 12th math scores with school name
ninth_math_scores = ninth.groupby(["school_name"]).mean()["math_score"]
tenth_math_scores = tenth.groupby(["school_name"]).mean()["math_score"]
eleventh_math_scores = eleventh.groupby(["school_name"]).mean()["math_score"]
twelfth_math_scores = twelfth.groupby(["school_name"]).mean()["math_score"]

# Using Pandas, create a data frame from our math grades
math_scores_by_grade = pd.DataFrame({
    "9th": ninth_math_scores,
    "10th": tenth_math_scores,
    "11th": eleventh_math_scores,
    "12th": twelfth_math_scores
    })

# Name columns and remove 'school_name' by removing index
math_scores_by_grade = math_scores_by_grade[["9th", "10th", "11th", "12th"]]
math_scores_by_grade.index.name = None

math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [36]:
# Redo same operations for Math Scores by Grade with Reading Score

# Breakdown school data by 9th, 10th, 11th, 12th
ninth = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth = school_data_complete[(school_data_complete["grade"] == "12th")]

# Using the variables created in the above code, create 9th, 10th, 11th, 12th math scores with school name
ninth_reading_scores = ninth.groupby(["school_name"]).mean()["reading_score"]
tenth_reading_scores = tenth.groupby(["school_name"]).mean()["reading_score"]
eleventh_reading_scores = eleventh.groupby(["school_name"]).mean()["reading_score"]
twelfth_reading_scores = twelfth.groupby(["school_name"]).mean()["reading_score"]

# Using Pandas, create a data frame from our reading grades
reading_scores_by_grade = pd.DataFrame({
    "9th": ninth_read_scores,
    "10th": tenth_read_scores,
    "11th": eleventh_read_scores,
    "12th": twelfth_read_scores
    })

# Name columns and remove 'school_name' by removing index
reading_scores_by_grade = math_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [51]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# Use our bin documenation to add another column to our latest dataframe (perc_school_summary)
# ie df[" "] = pd.cut(df[" "], bins, labels=group_names)
# add column to our perc_school_summary called [Spending Ranges (Per Student)]
perc_school_summary["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)
#perc_school_summary.head()

# Find the average spent on each student based on Average Math Score, 
# Average Reading Score, % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)
avg_math_spend = perc_school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
avg_reading_spends = perc_school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
perc_pass_math_spend = perc_school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
perc_pass_reading_spend = perc_school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_pass_spend = (perc_pass_math_spend + perc_pass_reading_spend) / 2

# Using Pandas, create a dataframe with the above calculated variables
school_spending = pd.DataFrame({
    "Average Math Score" : avg_math_spend,
    "Average Reading Score": avg_reading_spends,
    "% Passing Math": perc_pass_math_spend,
    "% Passing Reading": perc_pass_reading_spend,
    "% Overall Passing Rate": overall_pass_spend
})

# Clean the data, indexing
school_spending = school_spending[["Average Math Score", 
                                     "Average Reading Score", 
                                     "% Passing Math", 
                                     "% Passing Reading",
                                     "% Overall Passing Rate"]]

# Format data
school_spending["Average Math Score"] = school_spending["Average Math Score"].map("{:.2f}%".format)
school_spending["Average Reading Score"] = school_spending["Average Reading Score"].map("{:.2f}%".format)
school_spending["% Passing Math"] = school_spending["% Passing Math"].map("{:.2f}%".format)
school_spending["% Passing Reading"] = school_spending["% Passing Reading"].map("{:.2f}%".format)
school_spending["% Overall Passing Rate"] = school_spending["% Overall Passing Rate"].map("{:.2f}%".format)

school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.46%,83.93%,93.46%,96.61%,95.04%
$585-615,83.60%,83.89%,94.23%,95.90%,95.07%
$615-645,79.08%,81.89%,75.67%,86.11%,80.89%
$645-675,77.00%,81.03%,66.16%,81.13%,73.65%


In [45]:
perc_school_summary.head()
perc_school_summary.dtypes

School Type                        object
Total Students                      int64
Total School Budget               float64
Per Student Budget                float64
Average Math Score                float64
Average Reading Score             float64
% Passing Math                    float64
% Passing Reading                 float64
% Overall Passing Rate            float64
Spending Ranges (Per Student)    category
dtype: object

## Scores by School Size

* Perform the same operations as above, based on school size.

In [52]:
# Using the same process of adding a column, binning, creating a data frame
# Use same operations as a guideline for school size

# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Use our bin documenation to add another column to our latest dataframe (perc_school_summary)
# ie df[" "] = pd.cut(df[" "], bins, labels=group_names)
# add column to our perc_school_summary called [Spending Ranges (Per Student)]
perc_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=group_names)

# Find the average size of each school based on Average Math Score, 
# Average Reading Score, % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)
math_avg_size = perc_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
reading_avg_size = perc_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
perc_math_pass_size = perc_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
perc_reading_pass_size = perc_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
overall_pass_rate = (perc_math_pass_size + perc_reading_pass_size) / 2

# Using Pandas, create a dataframe with the above calculated variables
school_size = pd.DataFrame({
    "Average Math Score" : math_avg_size,
    "Average Reading Score": reading_avg_size,
    "% Passing Math": perc_math_pass_size,
    "% Passing Reading": perc_reading_pass_size,
    "% Overall Passing Rate": overall_pass_rate
})

# # Clean the data, indexing
school_size = school_size[["Average Math Score", 
                             "Average Reading Score", 
                             "% Passing Math", 
                             "% Passing Reading",
                             "% Overall Passing Rate"]]

# Format data
school_size["Average Math Score"] = school_size["Average Math Score"].map("{:.2f}%".format)
school_size["Average Reading Score"] = school_size["Average Reading Score"].map("{:.2f}%".format)
school_size["% Passing Math"] = school_size["% Passing Math"].map("{:.2f}%".format)
school_size["% Passing Reading"] = school_size["% Passing Reading"].map("{:.2f}%".format)
school_size["% Overall Passing Rate"] = school_size["% Overall Passing Rate"].map("{:.2f}%".format)

school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.82%,83.93%,93.55%,96.10%,94.82%
Medium (1000-2000),83.37%,83.86%,93.60%,96.79%,95.20%
Large (2000-5000),77.75%,81.34%,69.96%,82.77%,76.36%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [60]:
perc_school_summary["School Type"].unique()

array(['District', 'Charter'], dtype=object)

In [54]:
# Using the same process of adding a column, creating a data frame
# Use same operations as a guideline for school type

# Find the average size of each school based on Average Math Score, 
# Average Reading Score, % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)
math_avg_type = perc_school_summary.groupby(["School Type"]).mean()["Average Math Score"]
reading_avg_type = perc_school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
perc_math_pass_type = perc_school_summary.groupby(["School Type"]).mean()["% Passing Math"]
perc_reading_pass_type = perc_school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
overall_pass_type = (perc_math_pass_type + perc_reading_pass_type) / 2

# Using Pandas, create a dataframe with the above calculated variables
school_type = pd.DataFrame({
    "Average Math Score" : math_avg_type,
    "Average Reading Score": reading_avg_type,
    "% Passing Math": perc_math_pass_type,
    "% Passing Reading": perc_reading_pass_type,
    "% Overall Passing Rate": overall_pass_type
})

# # Clean the data, indexing
school_type = school_type[["Average Math Score", 
                             "Average Reading Score", 
                             "% Passing Math", 
                             "% Passing Reading",
                             "% Overall Passing Rate"]]



# Format data
school_type["Average Math Score"] = school_type["Average Math Score"].map("{:.2f}%".format)
school_type["Average Reading Score"] = school_type["Average Reading Score"].map("{:.2f}%".format)
school_type["% Passing Math"] = school_type["% Passing Math"].map("{:.2f}%".format)
school_type["% Passing Reading"] = school_type["% Passing Reading"].map("{:.2f}%".format)
school_type["% Overall Passing Rate"] = school_type["% Overall Passing Rate"].map("{:.2f}%".format)

school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47%,83.90%,93.62%,96.59%,95.10%
District,76.96%,80.97%,66.55%,80.80%,73.67%
